<a href="https://colab.research.google.com/github/SirTee12/LLM-Kaggle-Google-GenAI/blob/main/Prompt_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the SDK

In [4]:
!pip uninstall -qqy jupyterlab # uninstall jupyterlab in the case it conflict google colab base image
!pip install -U -q "google-genai==1.7.0" # install google genai library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.2 MB/s eta 0:00:00


import the SDK and some helpers for rendering the output. The `types` in `import types` may include custom data structures, classes or type hints psecifically to work with google AI. These custom types might represent things like model parameters, input/output formats for AI models or config settings.

In [1]:
from google import genai
from google.genai import types
from google.colab import userdata

from IPython.display import HTML, Markdown, display

## API Call Retry Implementation
Set up a retry helper. This allows you to "Run all" without worrying about per-minute quota


1.  **Imports `retry`:** It imports the `retry` module from `google.api_core`, which provides tools for automatic retries of API calls.

2.  **Defines `is_retriable`:**
    * It creates a lambda function called `is_retriable`.
    * This function checks if an exception (`e`) is an instance of `genai.errors.APIErrpr` and if its error code (`e.code`) is either 429 (Too Many Requests) or 503 (Service Unavailable).
    * It returns `True` if both conditions are met (meaning the API call is considered retriable), and `False` otherwise.

3.  **Applies Retry Logic:**
    * It modifies the `genai.models.Models.generate_content` function by wrapping it with the `retry.retry` function.
    * The `retry.retry` function is configured with the `is_retriable` function as the `predicate`. This tells `retry.retry` to only retry the API call if the `is_retriable` function returns `True` for the encountered exception.
    * Essentially, this replaces the original `generate_content` function with a new version that automatically retries on 429 and 503 errors, making the code more robust to temporary API issues.

In [2]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in (429, 503))

genai.models.Models.generate_content = retry.Retry(predicate = is_retriable)(genai.models.Models.generate_content)

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("secret-label-prompt")

## Get the API keys and create a client

Retrieves the API key securely and create a client object that will be used to communicate with the generative AI service, using the retrieved API key for authentication.

In [5]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # retrive the APi key
client = genai.Client(api_key=GOOGLE_API_KEY)   # create a client to interact with the genai application

## Running the First Prompt

In [6]:
# generate a response
response = client.models.generate_content(
    model = 'gemini-2.0-flash',
    contents = "Explain AI to me like I'm a kid"
)

print(response.text)
Markdown(response.text)

Imagine you have a really smart toy robot! That robot can do things like:

*   **Learn from playing:** If you play a game with the robot and show it what's good and bad, it can learn how to play better all by itself, just like you learn new games!
*   **Understand what you say:** You can talk to the robot, and it can understand what you mean, even if you don't say it perfectly. It's like having a friend who always knows what you're trying to say!
*   **Make guesses about the future:** If you show the robot lots of pictures of cats and dogs, it can learn to tell the difference and guess whether a new picture is a cat or a dog.

AI (Artificial Intelligence) is like giving that robot a brain! It's about making computers and robots smart enough to do things that usually only people can do, like learning, understanding, and making decisions.

It's not magic, though! People write special instructions (called code) to teach the computer or robot how to do these things. And the more it learns,

Imagine you have a really smart toy robot! That robot can do things like:

*   **Learn from playing:** If you play a game with the robot and show it what's good and bad, it can learn how to play better all by itself, just like you learn new games!
*   **Understand what you say:** You can talk to the robot, and it can understand what you mean, even if you don't say it perfectly. It's like having a friend who always knows what you're trying to say!
*   **Make guesses about the future:** If you show the robot lots of pictures of cats and dogs, it can learn to tell the difference and guess whether a new picture is a cat or a dog.

AI (Artificial Intelligence) is like giving that robot a brain! It's about making computers and robots smart enough to do things that usually only people can do, like learning, understanding, and making decisions.

It's not magic, though! People write special instructions (called code) to teach the computer or robot how to do these things. And the more it learns, the smarter it gets!


## Let's start a chat

In [7]:
chat = client.chats.create(model = "gemini-2.0-flash", history = [])
response = chat.send_message('Hello, my name is Ahmad')
print(response.text)

Hello Ahmad, it's nice to meet you! How can I help you today?



In [8]:
response = chat.send_message("can you tell me something interesting about dinosaurs")
print(response.text)

Okay, here's an interesting fact about dinosaurs that often surprises people:

**Some dinosaurs had feathers and might have even been warm-blooded like birds!**

For a long time, we thought of dinosaurs as scaly, cold-blooded reptiles. However, discoveries in recent decades, especially in China, have revealed fossil evidence of feathered dinosaurs. These feathers weren't necessarily for flight (although some were), but more likely for insulation, display, or even brooding eggs.

Think about that: some of those terrifying predators or giant herbivores might have been fluffy! This discovery has drastically changed our understanding of dinosaur evolution and their connection to modern birds.



In [9]:
response = chat.send_message("do you still remember my name")
print(response.text)

Yes, I remember your name is Ahmad. I'm designed to retain information from our conversation to help me better understand and respond to your requests.



# Chose a model from the Gemini model family

Retrieves a list of available AI models, searches for a specific model named "gemini-2.0-flash," and if found, displays its details in a formatted JSON-like structure. The search stops once the target model is located.

In [12]:
from pprint import pprint

for model in client.models.list():
  if model.name == "models/gemini-2.0-flash":
    pprint(model.to_json_dict())
    break

{'description': 'Gemini 2.0 Flash',
 'display_name': 'Gemini 2.0 Flash',
 'input_token_limit': 1048576,
 'name': 'models/gemini-2.0-flash',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent', 'countTokens'],
 'tuned_model_info': {},
 'version': '2.0'}


## Parameters Exploration

### Output Length

In [16]:
# set the max number of output token to 200
short_config = types.GenerateContentConfig(max_output_tokens = 200)

# create a response
response_short = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = short_config,
    contents='write a short essay on the importance of education in modern day society.'
)

print(response_short.text)

## The Cornerstone of Progress: The Indispensable Role of Education in Modern Society

In the ever-evolving landscape of the 21st century, education stands as the cornerstone of progress, the bedrock upon which individual fulfillment, societal advancement, and global stability are built. Its importance transcends the mere acquisition of knowledge, encompassing the development of critical thinking, adaptability, and responsible citizenship, all vital components for navigating the complexities of modern life.

Firstly, education empowers individuals. It equips them with the skills and knowledge necessary to participate meaningfully in the workforce, fostering economic independence and upward mobility. A well-educated populace is more likely to be innovative, creative, and adaptable to the rapidly changing demands of the job market. Furthermore, education expands horizons, cultivates intellectual curiosity, and promotes personal growth. It allows individuals to understand the world around

### Temperature

How much randomness is included when selecting the next word (token) in language models depends on the "temperature" parameter.  A higher temperature causes the model to take into account a greater number of potential words, producing more inventive and diverse results.  On the other hand, a lower temperature forces the model to adhere to the most likely terms, producing language that is more concentrated and predictable.  The model turns completely deterministic when the temperature is set to 0, always choosing the word that is the most likely.  Temperature does not, however, ensure actual randomness; rather, it is a means of directing the model toward more or less random results.

In [17]:
# create a variable to set the temperature
high_temp_config = types.GenerateContentConfig(temperature = 2.0)

for _ in range(5):
  response_temp = client.models.generate_content(
      model = 'gemini-2.0-flash',
      config = high_temp_config,
      contents = 'Pick a random colour... (respond in a single word)'
  )

  if response_temp.text:
    print(response_temp.text, '_' * 25)

Azure
 _________________________
Orange
 _________________________
Orange
 _________________________
Magenta
 _________________________
Azure
 _________________________
